In [1]:
!pip install -U openmim
!mim install mmengine
!mim install 'mmcv>=2.0.0rc1'
!mim install 'mmdet>=3.0.0rc0'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 372.1/372.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.9/200.9 kB 17.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 MB 6.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
     ━━━━━━━━━━━━━━━━

In [2]:
# !git clone https://github.com/open-mmlab/mmocr.git
!git clone https://github.com/PaulowniaTree/mini_project_4.git
%cd mini_project_4
!git checkout mmocr_custom
%cd mmocr_custom
!pip install -v -e .
# "-v" increases pip's verbosity.
# "-e" means installing the project in editable mode,
# That is, any local modifications on the code will take effect immediately.

Cloning into 'mini_project_4'...
remote: Enumerating objects: 2229, done.
remote: Counting objects: 100% (1170/1170), done.
remote: Compressing objects: 100% (858/858), done.
remote: Total 2229 (delta 274), reused 1166 (delta 270), pack-reused 1059
Receiving objects: 100% (2229/2229), 250.44 MiB | 29.03 MiB/s, done.
Resolving deltas: 100% (295/295), done.
/content/mini_project_4
Updating files: 100% (1949/1949), done.
Branch 'mmocr_custom' set up to track remote branch 'mmocr_custom' from 'origin'.
Switched to a new branch 'mmocr_custom'
/content/mini_project_4/mmocr_custom
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mini_project_4/mmocr_custom
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-i7m3s38z/mmocr.egg-info
  writing /tmp/pip-pip-egg-info-i7m3s38z/mmocr.egg-info/PKG-INFO
  

In [5]:
from mmocr.datasets import OCRDataset
from mmengine.registry import init_default_scope
from mmocr.apis import MMOCRInferencer
from mmengine.runner import Runner
import time
import mmcv
import matplotlib.pyplot as plt
from mmengine.runner import Runner
from mmengine import Config
from mmocr.apis import MMOCRInferencer

In [7]:
%pwd
%cd /content/mini_project_4/mmocr_custom/projects/ABCNet

/content/mini_project_4/mmocr_custom/projects/ABCNet


In [8]:
# root = 'C:/Users/thffh/Documents/project/project_4/mini_project_4/'
root = '/content/mini_project_4/'

In [9]:
cfg = Config.fromfile('config/abcnet/abcnet_resnet50_fpn_500e_icdar2015.py')

In [10]:
cfg.keys()

dict_keys(['num_classes', 'strides', 'bbox_coder', 'with_bezier', 'norm_on_bbox', 'use_sigmoid_cls', 'dictionary', 'model', 'test_pipeline', 'train_pipeline', 'icdar2015_textspotting_data_root', 'icdar2015_textspotting_train', 'icdar2015_textspotting_test', 'default_scope', 'env_cfg', 'randomness', 'default_hooks', 'log_level', 'log_processor', 'load_from', 'resume', 'val_evaluator', 'test_evaluator', 'vis_backends', 'visualizer', 'optim_wrapper', 'train_cfg', 'val_cfg', 'test_cfg', 'param_scheduler', 'train_dataloader', 'val_dataloader', 'test_dataloader', 'custom_imports', 'find_unused_parameters'])

In [11]:
cfg.train_dataloader.num_workers=1
cfg.val_dataloader.num_workers=1
cfg.test_dataloader.num_workers=1

In [12]:
cfg.dictionary = root+'mmocr_custom/dicts/kor_digits2.txt'
cfg.work_dir = 'work_dirs/test1/'
cfg.model.roi_head.rec_head.decoder.dictionary.dict_file = root+'mmocr_custom/dicts/kor_digits2.txt'

In [13]:
cfg.train_cfg.max_epochs=1
cfg.train_cfg.val_interval=1

In [14]:
cfg.icdar2015_textspotting_data_root = root+'Temp/temp_Sign'
cfg.icdar2015_textspotting_train.data_root = root+'Temp/temp_Sign'
cfg.icdar2015_textspotting_train.ann_file = root+'Temp/spot_train_annotation.json'
cfg.icdar2015_textspotting_test.data_root = root+'Temp/temp_Sign'
cfg.icdar2015_textspotting_test.ann_file = root+'Temp/spot_test_annotation.json'

In [15]:
cfg.train_dataloader.batch_size=1
cfg.train_dataloader.dataset.data_root = root+'Temp/temp_Sign'
cfg.train_dataloader.dataset.ann_file = root+'Temp/spot_train_annotation.json'

In [16]:
cfg.val_dataloader.batch_size=1
cfg.val_dataloader.dataset.data_root = root+'Temp/temp_Sign'
cfg.val_dataloader.dataset.ann_file = root+'Temp/spot_test_annotation.json'

In [17]:
cfg.test_dataloader.batch_size=1
cfg.test_dataloader.dataset.data_root = root+'Temp/temp_Sign'
cfg.test_dataloader.dataset.ann_file = root+'Temp/spot_test_annotation.json'

In [19]:
cfg.keys()

dict_keys(['num_classes', 'strides', 'bbox_coder', 'with_bezier', 'norm_on_bbox', 'use_sigmoid_cls', 'dictionary', 'model', 'test_pipeline', 'train_pipeline', 'icdar2015_textspotting_data_root', 'icdar2015_textspotting_train', 'icdar2015_textspotting_test', 'default_scope', 'env_cfg', 'randomness', 'default_hooks', 'log_level', 'log_processor', 'load_from', 'resume', 'val_evaluator', 'test_evaluator', 'vis_backends', 'visualizer', 'optim_wrapper', 'train_cfg', 'val_cfg', 'test_cfg', 'param_scheduler', 'train_dataloader', 'val_dataloader', 'test_dataloader', 'custom_imports', 'find_unused_parameters', 'work_dir'])

In [20]:
cfg.dictionary

'/content/mini_project_4/mmocr_custom/dicts/kor_digits2.txt'

In [18]:
cfg.visualizer.name = f'{time.localtime()}'

runner = Runner.from_cfg(cfg)

05/18 06:56:32 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.11 (main, Apr  5 2023, 14:15:10) [GCC 9.4.0]
    CUDA available: False
    numpy_random_seed: 1179561532
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.8, CUDNN_VERSION=8.7.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-deprecated 

TypeError: ignored

In [ ]:
runner.train()